In [ ]:
from typing import Annotated, List, Dict
from typing_extensions import TypedDict
from langgraph.graph.message import add_messages
from langchain_core.messages import AIMessage
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_openai import ChatOpenAI
from langchain_teddynote.tools.tavily import TavilySearch
from langgraph.graph import StateGraph, START, END
from langgraph.prebuilt import ToolNode, tools_condition
from langgraph.checkpoint.memory import MemorySaver
from dotenv import load_dotenv
from langchain_core.runnables import RunnableConfig
import yaml
from pathlib import Path


load_dotenv()

True

In [2]:
class State(TypedDict):
    messages: Annotated[list, add_messages]

In [3]:
tavily_search = TavilySearch(max_results=3)
designer_tools = [tavily_search]

In [ ]:
def load_system_template(yaml_path: str) -> str:
    config_path = Path(yaml_path)
    prompt_data = yaml.safe_load(config_path.read_text(encoding='utf-8'))
    return prompt_data['template']

In [ ]:
def designer(state: State) -> State:

    system_template = load_system_template()

    prompt = ChatPromptTemplate.from_messages(
        [
            ('system', system_template),
            MessagesPlaceholder(variable_name='messages'),
        ]
    )
    llm = ChatOpenAI(model='gpt-4o-mini')
    llm_with_tools = llm.bind_tools(designer_tools)
    chain = prompt | llm_with_tools
    return {'messages': [chain.invoke({'messages': state['messages']})]}

In [11]:
graph_builder = StateGraph(State)
designer_tool_node = ToolNode(tools=designer_tools)
graph_builder.add_node("designer", designer)
graph_builder.add_node("designer_tool", designer_tool_node)

# Add edges
graph_builder.add_edge(START, "designer")
graph_builder.add_conditional_edges(
    "designer",
    tools_condition,
    {"tools": "designer_tool", "__end__": END},
)

graph_builder.add_edge("designer", END)

memory = MemorySaver()
graph = graph_builder.compile(checkpointer=memory)

In [12]:
question = "옥수수 수염차 홍보물 만들어줘"

config = RunnableConfig(recursion_limit=10, configurable={"thread_id": "1"})
# Stream and display messages
for event in graph.stream({"messages": [("user", question)]}, config=config):
    for value in event.values():
        value["messages"][-1].pretty_print()
# Final output
state = graph.get_state({"configurable": {"thread_id": "1"}})
final_output = state.values["messages"][-1].content

================================== Ai Message ==================================

아래는 "옥수수 수염차" 홍보물을 위한 디자인 기획안입니다.

# 1. 브리프 요약
- **핵심 가치**: 옥수수 수염차의 자연성, 건강한 선택의 용이성, 자연에서 온 해독 효과  
- **홍보 목적**: 소비자에게 옥수수 수염차의 이점 및 효능을 알리고, 제품 구매 유도  
- **기대 효과**: 인지도 향상, 브랜드 이미지 강화, 판매 증대  

# 2. 타겟 오디언스 & 톤·매너
- **타겟 고객**: 20대 후반에서 40대 초반의 건강과 웰빙에 관심이 많은 여성  
- **라이프스타일**: 자연 식품을 선호하며, 유기농 및 건강한 음료를 적극적으로 찾는 소비자  
- **톤·무드**: 자연적이고 건강하며 따뜻한 느낌의 프리미엄 브랜드 이미지   

# 3. 디자인 컨셉 기획
- **핵심 메시지**: “자연이 선사하는 건강한 디톡스”  
- **비주얼 키워드**: 옥수수 수염, 자연, 선명한 색감, 청량감  
- **스타일 방향**: 플랫 일러스트와 자연 풍경을 혼합한 현대적 디자인  

# 4. 컬러·타이포·그래픽 가이드
- **추천 컬러 팔레트**: 
  - #FFC107 (옥수수 노란색)
  - #4CAF50 (자연초록)
  - #FFFFFF (화이트)
- **서체 가이드**:
  - 헤드라인: Noto Sans KR Bold
  - 본문: Noto Serif KR Regular
- **아이콘·일러스트 스타일**: 풀 컬러의 일러스트와 선명한 아이콘 사용  

# 5. 레이아웃 & 정보구조
- **AIDA 프레임워크 적용**:
  - Attention: 주목을 끌 수 있는 비주얼과 헤드라인
  - Interest: 제품 특징 설명 및 텍스트 섹션
  - Desire: 소비자 후기 및 효과 강조
  - Action: 구매 버튼 및 웹사이트 링크 
- **주요 섹션 배치**: 
  - 헤드라인 (상단 중앙)
